In [35]:
import numpy as np
import pandas as pd
# https://www.youtube.com/watch?v=QJoa0JYaX1I 16:20

In [54]:
df = pd.read_csv("xOR_problem.csv").loc[:,'0':'0.2']
X_train = df.loc[:,'0':'0.1']
Y_train = df.loc[:,'0.2']
print(df)
print(" ")


   0  0.1  0.2
0  1    1    0
1  0    1    1
2  1    0    1
3  0    0    0
 


In [182]:
## Need to generalize it for n layers! it is generalized for n nodes in the 3 layers.
class nn:
    input_nodes = 0
    hidden_nodes = 0
    output_nodes = 0
    
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.input_nodes = input_nodes # The number of features!
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        self.weights_ih = np.random.rand(self.hidden_nodes, self.input_nodes)
        self.weights_ho = np.random.rand(self.output_nodes, self.hidden_nodes)
        self.bias_h = np.empty([self.hidden_nodes, 1])
        self.bias_o = np.empty([self.output_nodes, 1])
        self.learning_rate = 0.1

    def sigmoid(self,x):
        res = 1 / (1 + np.exp(-x))
        return res
    
    def feedforward(self, input_sample):
        # Generating the hidden outputs:
        hidden = np.matrix(np.dot(self.weights_ih, input_sample)).T
        hidden = np.add(hidden,self.bias_h)
        sig = lambda t: self.sigmoid(t)
        hidden = sig(hidden)
        
        # Now, generate the output for the output layer:
        outputs = np.matrix(np.dot(self.weights_ho, hidden ))
        outputs = np.add(outputs,self.bias_o)
        sig = lambda t: self.sigmoid(t)
        outputs = sig(outputs)
        output_outputs = np.asmatrix(outputs)

        return hidden, output_outputs
    
    def train(self, input_sample, targets):
        targets = np.asmatrix(targets).T
        
        hidden, outputs =  self.feedforward(input_sample)
    
        # Calculate the errors from the output layer:
        # ERROR = Targets - ypred
        output_errors = np.subtract(targets,outputs)
    
        # Calculate gradient
        # gradient = outputs*(1 - outputs)
        desig = lambda t: self.desigmoid(t)
        gradients = desig(np.asmatrix(outputs))
        gradients = np.multiply(gradients, output_errors)
        gradients = np.multiply(gradients, self.learning_rate)
        
        # Calculate deltas
        hidden_T = hidden.T
        weight_deltas_ho = np.dot(gradients, hidden_T)
        
        # Adjust the weights by the deltas
        self.weights_ho = np.add(self.weights_ho,weight_deltas_ho)
 
        # Adjust the bias by it's deltas:
        self.bias_h = np.add(self.bias_o, gradients)        
 
        # Calculate the error from the hidden node:
        hidden_errors = np.dot(self.weights_ho.T,output_errors)
        
        # Calculate the gradient for the hidden layer:
        hidden_gradient = desig(np.asmatrix(hidden))
        hidden_gradient = np.multiply(hidden_gradient, hidden_errors)
        hidden_gradient = np.multiply(hidden_gradient, self.learning_rate)
        
        # Calculate change of weight for input -> hidden (deltas):
        input_sample_T = np.matrix(input_sample)
        weight_ih_deltas = np.dot(hidden_gradient, input_sample_T)
        
        # Update the input -> Hidden weights:
        self.weights_ih = np.add(self.weights_ih, weight_ih_deltas)
        
        # Adjust the bias by it's deltas:
        self.bias_h = np.add(self.bias_h, hidden_gradient)
        

        
    def desigmoid(self, y):
        res = y.T*(1-y)
        return res
    
# ********* Main ***********
# The number of nodes in the neural net:
n1 = nn(2,2,1)

x_points=[]
for row in df.iterrows():
    index, data = row
    data = data.tolist()
    y_train = [data.pop(2)]
    x_train = data
    n1.train(x_train, y_train)




[1, 1] [0]
n1.weights_ih
[[0.86793337 0.14236718]
 [0.20625149 0.34055009]]
 
[0, 1] [1]
n1.weights_ih
[[0.86793337 0.15569544]
 [0.20625149 0.34315054]]
 
[1, 0] [1]
n1.weights_ih
[[0.87926786 0.15569544]
 [0.20851642 0.34315054]]
 
[0, 0] [0]
n1.weights_ih
[[0.87926786 0.15569544]
 [0.20851642 0.34315054]]
 


In [183]:
h, o = n1.feedforward([1,1])
print(o)

h, o = n1.feedforward([0,1])
print(o) 

h, o = n1.feedforward([1,0])
print(o) 

h, o = n1.feedforward([0,0])
print(o) 

[[0.65357134]]
[[0.61932516]]
[[0.64597376]]
[[0.61007728]]
